## Le script synthèse et traduction du LADIREC
Il utilise le cadre de travail de Hugging face ou sont déposés des modèles de langues pour tiré de différentes stratégies.
Ce script utilise les modèles de type BERT et applique les contraintes de ce type de modèle (longeur maximale, type de lemmatisation etc.).
Il est possible de modifier le modèle utilisé à même le code.

## Import des librairies

In [ ]:
# Le premier bloc doit être roulé en mode administrateur
# Installation des librairies et packages nécessaire au script
# Cette opération ne devrait être exécutée qu'une seule fois (ou à même le terminal), mais peut-être effectuer plusieurs foies sans conséquence.
!python -m pip install --upgrade pip setuptools notebook jupyter

# https://pytorch.org/get-started/locally/ to get the pyTorch command line to install on you local machine use this link
# the command provider here is for windows/pip/python/CUDA11.6
# link to download the latest windows CUDA toolkit https://developer.nvidia.com/cuda-downloads
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers[sentencepiece] sacremoses

!pip install pandas openpyxl

# Documentation de Pipeline sur huggingFace : https://huggingface.co/docs/transformers/v4.22.1/en/quicktour#pipeline

In [ ]:
# Import des librairies
import pandas as pd
import time
from transformers import pipeline

## Déclaration des variables globales, constantes et paramêtres du traitement

In [ ]:
# PATH_CORPUS contient le chemin et le nom du fichier original du corpus - format xlsx demandé
# Cette valeur est une constante et ne devrait pas changer
PATH_CORPUS = "corpus_copie_fr_short.xlsx"

# LISTE_COLONNE correspond à la liste des noms des colonnes sélectionnées pour les analyses
# !cette valeur est une constante et ne devrait pas changer
LISTE_COLONNE = ['T1','U1']

# LANGUE_OG_CORPUS correspond à la langue original du corpus de texte lors du READ - 
# Cette valeur est une constante et ne devrait pas changer en cours de traitement
LANGUE_OG_CORPUS = 'FR'

# LANGUE_CHAINE dresse la liste des langues pour la traduction en chaine (traduction dans une/des langues, puis retour à la langue originale LANGUE_OG_CORPUS)
# Cette valeur est une constante et ne devrait pas changer
LANGUE_CHAINE = ['ES', 'EN']

# utiliser comme paramètre de summerizer pour la longueur maximum du résumé
SYNTHESE_MAX_LENGTH = 80

# nombre de mot dans un chunk
CHUNK_SIZE = 250


## Déclaration des fonctions utilisées

In [ ]:
# Chargement du corpus
# la fonction qui importe et créé le dataset à partir du corpus
def get_dataset(path, col = None):
    df1 = pd.read_excel(path, usecols=col)
    #par défaut, toutes le colonnes de la table sont chargé en df1

    return df1


In [ ]:
# Fusion des colonnes à analyser
# la fonction qui fusionne des colonnes selon une liste sélectionnée dans la variable globale LISTE_COLONNE.
def merge_col(df, colSelect):

    df_merged = pd.DataFrame(columns=['raw'])
    df_merged['raw'] = df[colSelect].apply(
        lambda row: (" ".join(row.values.astype(str))), axis=1
    )
    # # Ajouté : df_merged[raw] qui contient la fusion des colonnes sélectionnées sans le retrait des majuscules, le df retourné contient 2 col.
    # df_merged['line'] = df[colSelect].apply(
    #     lambda row: (" ".join(row.values.astype(str)).lower()), axis=1
    # )

    # display(df_merged)
    return df_merged

## Début du traitement

In [ ]:
# chargement de données (corpus)
# MAIN
print(PATH_CORPUS)
start_time = time.time()

data_origin = get_dataset(PATH_CORPUS)
data = merge_col(data_origin, LISTE_COLONNE)
data["Doc_ID"] = data_origin["Doc_ID"]

print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)


In [ ]:
# chunk the raw data in segment of CHUNK_SIZE mots
from cgitb import text


def text_chunking(data, chunk_size)
    start_time = time.time()

    nb_word_series = []
    raw_series = data['raw']
    raw_chunk_series = []
    chunks = []
    continue_chunking = True

    for index, segment in raw_series.items():
        nb_word_series.append(len(segment.split(' ')))
        # print(f"Nombre de mot {index}: {len(segment.split(' '))}")

        while continue_chunking:
            tokens = segment.split(" ")
            if len(tokens) >= CHUNK_SIZE:
                index_dernier_point = " ".join(tokens[:CHUNK_SIZE]).rfind(".") + 1
                if index_dernier_point > 0:
                    chunks.append(segment[:index_dernier_point])
                    segment = segment[index_dernier_point:]
                else:
                    chunks.append(" ".join(tokens[:CHUNK_SIZE]))
                    segment = " ".join(tokens[CHUNK_SIZE:])
            else:
                chunks.append(segment)
                continue_chunking = False

        raw_chunk_series.append(chunks)

        chunks = []
        continue_chunking = True

    # ajouter les text chunker dans une nouvel colonne du df
    data["raw_chunk"] = raw_chunk_series
    data["nb_word"] = nb_word_series

    print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
    display(data)
return data

In [ ]:
# chunk the raw data in segment of CHUNK_SIZE mots
start_time = time.time()

nb_word_series = []
raw_series = data['raw']
raw_chunk_series = []
chunks = []
continue_chunking = True

for index, segment in raw_series.items():
    nb_word_series.append(len(segment.split(' ')))
    # print(f"Nombre de mot {index}: {len(segment.split(' '))}")

    while continue_chunking:
        tokens = segment.split(" ")
        if len(tokens) >= CHUNK_SIZE:
            index_dernier_point = " ".join(tokens[:CHUNK_SIZE]).rfind(".") + 1
            if index_dernier_point > 0:
                chunks.append(segment[:index_dernier_point])
                segment = segment[index_dernier_point:]
            else:
                chunks.append(" ".join(tokens[:CHUNK_SIZE]))
                segment = " ".join(tokens[CHUNK_SIZE:])
        else:
            chunks.append(segment)
            continue_chunking = False

    raw_chunk_series.append(chunks)

    chunks = []
    continue_chunking = True

# ajouter les text chunker dans une nouvel colonne du df
data["raw_chunk"] = raw_chunk_series
data["nb_word"] = nb_word_series

print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)

In [ ]:
# Sauvegarder le corpus nettoyé dans un fichier excel
# à faire : réordonner les colonnes
# + ajouter une variable globale de noms de colonnes du dataset original à ajouter aux sorties à titre de parametrage

data.to_excel('corpus_nettoyé.xlsx',index=False)

## Recherche des entités nommées

In [ ]:
# Nouveau Pipeline avec hugging face à exploiter ici
# DEV

## Synthétiser des textes en français

In [ ]:
summarizer = pipeline("summarization",
                       model="moussaKam/barthez-orangesum-title")

In [ ]:
start_time = time.time()

# roule la synthese sur chaque chunk de chaque text
data['synthese'] = data['raw_chunk'].apply(
    lambda row: ([summarizer(chunk, max_length=SYNTHESE_MAX_LENGTH) for chunk in row])
)

# remettre ensemble les syntheses comme un grand string dans une seule colones
data['synthese'] = data['synthese'].apply(
    lambda row: (" | ".join([chunk[0]["summary_text"] for chunk in row]))
)

print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)

## Traduction

In [ ]:
# Préparation du pipeline de traduction de français à anglais
fr_to_en = pipeline("translation_fr_to_en", 
                    model="Helsinki-NLP/opus-mt-fr-en")

In [ ]:
start_time = time.time()

# roule la traduction de français à anglais sur chaque chunk de chaque texte
data['fr_to_en'] = data['raw_chunk'].apply(
    lambda row: (fr_to_en(row))
)

# remettre ensemble les syntheses comme un grand string dans une seule colones
data['fr_to_en'] = data['fr_to_en'].apply(
    lambda row: (" | ".join([chunk['translation_text'] for chunk in row]))
)
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)

In [ ]:
display(data)

In [ ]:
# Chargement du pipeline de traduction de anglais à français
en_to_fr = pipeline("translation_en_to_fr", 
                    model="Helsinki-NLP/opus-mt-en-fr")

In [ ]:
start_time = time.time()

# roule la traduction de anglais à français sur chaque chunk de chaque texte

data['en_to_fr'] = data['fr_to_en'].apply(
    lambda row: (en_to_fr(row))
)

# remettre ensemble les traductions comme un grand string dans une seule colones
data['en_to_fr'] = data['en_to_fr'].apply(
    lambda row: (" | ".join([chunk['translation_text'] for chunk in row]))
)
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)

## Export final

In [ ]:
# Sauvegarde du dataframe sous forme MS Excel 
# réordonner les colonnes
# ajouter une liste de nom de colonnes du DF original à ajouter à la sortie corpus enrichi

data.to_excel('corpus_enrichi.xlsx')